### Pre-requisites

In [27]:
import asyncio
from rich.pretty import pprint
from mcp import ClientSession
from mcp.client.sse import sse_client

URL = "http://localhost:8000/sse"

### Native Client

In [28]:
async with sse_client(URL) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()
        result = await session.list_tools()
        pprint(result.tools)

        result = await session.call_tool("list_tasks")
        pprint(result.content)

[
│   Tool(
│   │   name='list_tasks',
│   │   description='\n    List all tasks in the system.\n    ',
│   │   inputSchema={'properties': {}, 'type': 'object'},
│   │   annotations=None
│   ),
│   Tool(
│   │   name='add_task',
│   │   description='\n    Add a new task to the system.\n    ',
│   │   inputSchema={
│   │   │   'properties': {'name': {'title': 'Name', 'type': 'string'}},
│   │   │   'required': ['name'],
│   │   │   'type': 'object'
│   │   },
│   │   annotations=None
│   )
]

[
│   TextContent(
│   │   type='text',
│   │   text='[\n  {\n    "id": 1,\n    "name": "Eat breakfast",\n    "is_completed": true\n  },\n  {\n    "id": 2,\n    "name": "Go to work",\n    "is_completed": false\n  },\n  {\n    "id": 3,\n    "name": "Attend meetings",\n    "is_completed": false\n  },\n  {\n    "id": 4,\n    "name": "Read a book",\n    "is_completed": false\n  },\n  {\n    "id": 5,\n    "name": "Exercise",\n    "is_completed": false\n  }\n]',
│   │   annotations=None
│   )
]

### Langchain


#### Tools via ClientSession

In [10]:
from langchain_mcp_adapters.tools import load_mcp_tools

async with sse_client(URL) as (read_stream, write_stream):
    async with ClientSession(read_stream, write_stream) as session:
        await session.initialize()

        tools = await load_mcp_tools(session)
        pprint(tools)

[
│   StructuredTool(
│   │   name='list_tasks',
│   │   description='\n    List all tasks in the system.\n    ',
│   │   args_schema={'properties': {}, 'type': 'object'},
│   │   response_format='content_and_artifact',
│   │   coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002D0BB495D00>
│   ),
│   StructuredTool(
│   │   name='add_task',
│   │   description='\n    Add a new task to the system.\n    ',
│   │   args_schema={
│   │   │   'properties': {'name': {'title': 'Name', 'type': 'string'}},
│   │   │   'required': ['name'],
│   │   │   'type': 'object'
│   │   },
│   │   response_format='content_and_artifact',
│   │   coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002D0BB495C60>
│   )
]

#### via MultiServerMCPClient

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "mcp_server": {
            "transport": "sse",
            "url": URL
        },
    } # type: ignore
)
tools = await client.get_tools()
pprint(tools)

resources = await client.get_resources("mcp_server")
pprint(resources)

[
│   StructuredTool(
│   │   name='list_tasks',
│   │   description='\n    List all tasks in the system.\n    ',
│   │   args_schema={'properties': {}, 'type': 'object'},
│   │   response_format='content_and_artifact',
│   │   coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002D0BC173A60>
│   ),
│   StructuredTool(
│   │   name='add_task',
│   │   description='\n    Add a new task to the system.\n    ',
│   │   args_schema={
│   │   │   'properties': {'name': {'title': 'Name', 'type': 'string'}},
│   │   │   'required': ['name'],
│   │   │   'type': 'object'
│   │   },
│   │   response_format='content_and_artifact',
│   │   coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002D0BC172C00>
│   ),
│   StructuredTool(
│   │   name='complete_task',
│   │   description='\n    Mark a task as completed.\n    ',
│   │   args_schema={
│   │   │   'properties': {'task_id': {'title': 'Task Id', 'type': 'integer'}},
│   │   │   'required': ['task_id'],
│   │   │   'type': 'object'
│   │   },
│   │   response_format='content_and_artifact',
│   │   coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000002D0BC172F20>
│   )
]

[Blob(metadata={'uri': AnyUrl('config://version')}, data='1.0.0', mimetype='text/plain')]

### LLMing

#### Raw model

In [18]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen3:8b")
llm_with_tools = llm.bind_tools(tools)

In [19]:
response = llm_with_tools.invoke("what are the current tasks?")
pprint(response)

AIMessage(
│   content="<think>\nOkay, the user is asking about the current tasks. Let me check the available tools. There's a function called list_tasks that doesn't require any parameters. Since the user wants to know the existing tasks, I should call list_tasks. No need for add_task here because they're not adding anything. Just need to trigger the list_tasks function.\n</think>\n\n",
│   additional_kwargs={},
│   response_metadata={
│   │   'model': 'qwen3:8b',
│   │   'created_at': '2025-06-13T09:14:41.0387031Z',
│   │   'done': True,
│   │   'done_reason': 'stop',
│   │   'total_duration': 2822390200,
│   │   'load_duration': 156959600,
│   │   'prompt_eval_count': 195,
│   │   'prompt_eval_duration': 4554100,
│   │   'eval_count': 90,
│   │   'eval_duration': 2659029700,
│   │   'model_name': 'qwen3:8b'
│   },
│   id='run--cb15bedc-dad6-4ef6-895f-45d0a7e4cbef-0',
│   tool_calls=[
│   │   {'name': 'list_tasks', 'args': {}, 'id': '6c1aa204-33d2-4bcd-9abc-6486c161c4c4', 'type': 'tool_call'}
│   ],
│   usage_metadata={'input_tokens': 195, 'output_tokens': 90, 'total_tokens': 285}
)

In [26]:
# call tool manually
tc = response.tool_calls[-1]
result = await {tool.name: tool for tool in tools}[tc['name']].ainvoke(tc)
print(result.content)

[
  {
    "id": 1,
    "name": "Eat breakfast",
    "is_completed": true
  },
  {
    "id": 2,
    "name": "Go to work",
    "is_completed": false
  },
  {
    "id": 3,
    "name": "Attend meetings",
    "is_completed": false
  },
  {
    "id": 4,
    "name": "Read a book",
    "is_completed": false
  },
  {
    "id": 5,
    "name": "Exercise",
    "is_completed": false
  }
]


#### Agent

Agent handles calling tools and evaluating the results automatically.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model="ollama:qwen3:8b", # or we can specify the llm instance above
    tools=tools,
)

In [ ]:
# the tools are coroutines, so we need to use `ainvoke` instead of `invoke`
# if async methods are not used, invoke runs forever. 
response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "what are the current tasks?"}]}
)

In [ ]:
response

{'messages': [HumanMessage(content='what are the current tasks?', additional_kwargs={}, response_metadata={}, id='84c71b7b-8ee3-4239-94cf-c0fb7ce35697'),
  AIMessage(content="<think>\nOkay, the user is asking about the current tasks. Let me check the available tools. There's a function called list_tasks that doesn't require any parameters. Since the user wants to know the existing tasks, I should call list_tasks. No need for any arguments here. Just execute that function and return the result.\n</think>\n\n", additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-06-13T09:12:02.1259645Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19864783700, 'load_duration': 16376419900, 'prompt_eval_count': 195, 'prompt_eval_duration': 482851900, 'eval_count': 84, 'eval_duration': 3002409600, 'model_name': 'qwen3:8b'}, id='run--39aaa854-d155-48cd-9c3a-8c2a54b43e3b-0', tool_calls=[{'name': 'list_tasks', 'args': {}, 'id': 'b04c0b77-3736-4c07-ac8a-5a22287c40db', '